In [1]:
import pandas as pd
import numpy as np
import torch
from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast, AdamW
from tqdm import tqdm

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# CUDA 사용 가능 여부 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# 모델과 토크나이저 불러오기
model = AutoModelForCausalLM.from_pretrained('/content/drive/MyDrive/DBDBDeep/model_skt')
tokenizer = AutoTokenizer.from_pretrained('/content/drive/MyDrive/DBDBDeep/model_skt')

In [6]:
# Inference를 위한 test.csv 파일 로드
test = pd.read_csv('/content/drive/MyDrive/DBDBDeep/test.csv')

# test.csv의 '질문'에 대한 '답변'을 저장할 리스트
preds = []

In [7]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

# '질문' 컬럼의 각 질문에 대해 답변 생성
for test_question in tqdm(test['질문']):
    # 입력 텍스트를 토큰화하고 모델 입력 형태로 변환
    sent = '<usr>' + test_question + '<sys>'
    input_ids = [tokenizer.bos_token_id] + tokenizer.encode(sent, add_special_tokens=False)
    input_ids = torch.tensor([input_ids], dtype=torch.long).to(device)

    # 답변 생성
    output_sequences = model.generate(
        input_ids=input_ids.to(device),
        max_length=300,
        temperature=0.9,
        top_k=1,
        top_p=0.9,
        repetition_penalty=1.2,
        do_sample=True,
        num_return_sequences=1
    )

    # # 생성된 텍스트(답변) 출력
    # for generated_sequence in output_sequences:
    #     full_text = tokenizer.decode(generated_sequence, skip_special_tokens=False)
    #     # 질문과 답변의 사이를 나타내는 eos_token (</s>)를 찾아, 이후부터 출력
    #     answer_start = full_text.find(tokenizer.eos_token) + len(tokenizer.eos_token)
    #     answer_only = full_text[answer_start:].strip()
    #     answer_only = answer_only.replace('\n', ' ')
    #     sentence_without_end_token = sentence.replace('</s>', '')
    #     print(sentence_without_end_token)
    #     preds.append(answer_only)

    # 생성된 텍스트(답변) 출력
    for generated_sequence in output_sequences:
        full_text = tokenizer.decode(generated_sequence, skip_special_tokens=False)
        # 질문과 답변의 사이를 나타내는 eos_token (</s>)를 찾아, 이후부터 출력
        answer_start = full_text.find(tokenizer.eos_token) + len(tokenizer.eos_token)
        answer_only = full_text[answer_start:].strip()
        answer_only = answer_only.replace('\n', ' ')
        # '<usr>', '<sys>'와 '</s>'를 제거하여 정제된 답변을 얻기
        answer_only = answer_only.replace('<usr>', '').replace('<sys>', '').replace('</s>', '')
        print(answer_only)
        preds.append(answer_only)

  1%|          | 1/130 [00:08<17:25,  8.11s/it]

 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요? 방청 페인트는 내구성이 상대적으로 떨어져서 실외 사용이나 강한 마모에 노출될 경우 추가적인 보호코팅이 필요할 수 있습니다.


  2%|▏         | 2/130 [00:15<16:43,  7.84s/it]

 도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요? 도배지 안쪽의 길 형태의 타원으로 발생한 기포는 보수가가능합니다. 벽지면과 벽지면 사이가 벌어진 경우, 이것이 발생한 원인을 파악하기 위해 전문가의 조언을 받아야 합니다.


  2%|▏         | 3/130 [00:21<14:52,  7.03s/it]

...

 평지붕의 누수 문제를 방지하기 위해 수성 벽체용 탄성 방수 도료를 사용하는 것이 어떤 장점이 있나요? 벽지면과 벽지면 사이가 벌어졌을 때 주변 환경에 미치는 영향을 최소화하기 위해 빠른 조치가 필요합니다.


 99%|█████████▉| 129/130 [10:11<00:04,  4.68s/it]

 석고수정이 발생하는 가장 큰 원인은 무엇인가요? 그리고 이를 해결하는 방법에 대해 알려주세요." 벽을 이루고 있는 석고에 훼손이 생겼을 때 어떤 방법으로 신속히 복구시키는 것이 중요합니다. 뿐만 아니라, 건축물 내부의 습기, 먼지, 오염물질로부터 보호할 수 있도록 주의해야 합니다. 예를 들어, 얼룩이 생기는 것을 제거하고 제거하기 위해 벽지를 교체하는 것이 좋습니다.


100%|██████████| 130/130 [10:14<00:00,  4.73s/it]

 카페트의 기대 수명은 얼마나 될까요? 그리고 오리지널징크의 장점에는 무엇이 있나요? 카페트의 장점은 다양한 패턴과 디자인을 적용할 수 있다는 것입니다. 또한 친환경적이고 높은 내식성이 있어 수명이 오랫동안 유지될 수 있는 것이 있습니다.


In [ ]:
# !pip install sentence_transformers

In [8]:
from sentence_transformers import SentenceTransformer
import numpy as np

# 평가를 위한 Embedding Vector 추출에 활용할 모델(distiluse-base-multilingual-cased-v1) 불러오기
model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

# 생성한 모든 응답(답변)으로부터 Embedding Vector 추출
pred_embeddings = []

for pred in preds:
    # 각 응답(답변)을 임베딩
    embedding = model.encode(pred, convert_to_tensor=True)
    pred_embeddings.append(embedding)

pred_embeddings = np.array(pred_embeddings)
print(pred_embeddings.shape)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


(130, 512)


In [17]:
embedding

tensor([ 0.0286, -0.0370, -0.0195, -0.0373,  0.0559, -0.0344,  0.0004,  0.0255,
         0.0860, -0.0294, -0.0204,  0.0064, -0.0535, -0.0191, -0.0070,  0.0350,
         0.0049,  0.0448, -0.0567, -0.0571, -0.0064,  0.0174, -0.0243, -0.0351,
         0.0163,  0.0366,  0.0720, -0.0147, -0.0774,  0.0365,  0.0096,  0.0013,
        -0.0115,  0.0182, -0.0591,  0.0137,  0.0500, -0.0155, -0.0130, -0.0228,
        -0.0440,  0.0368, -0.0403,  0.1224,  0.0058,  0.0607, -0.0066, -0.0138,
         0.0425, -0.0214,  0.0148, -0.0074,  0.0458,  0.0781,  0.0392, -0.0158,
        -0.0111, -0.0022,  0.0328,  0.0427,  0.0110, -0.0152, -0.0334,  0.0150,
        -0.0515,  0.0479, -0.0640,  0.0573,  0.0711,  0.0113, -0.0051,  0.0413,
        -0.0489, -0.0175, -0.0413,  0.0763,  0.1081,  0.0201,  0.0821,  0.0410,
        -0.0300, -0.0966,  0.0230,  0.0439, -0.0776,  0.0336,  0.0169,  0.0379,
        -0.0051,  0.0567,  0.0102,  0.0237,  0.0482, -0.0530,  0.0151,  0.0056,
         0.0234,  0.0196,  0.0234,  0.03

In [18]:
pred_embeddings

array([[ 0.02060867,  0.05562091,  0.01614225, ..., -0.02704569,
        -0.03798091,  0.0684455 ],
       [-0.01330859, -0.0090784 , -0.00374858, ..., -0.03581328,
         0.00828904,  0.05989039],
       [ 0.03259891, -0.04804637, -0.0747473 , ..., -0.01113238,
        -0.03899511,  0.08017483],
       ...,
       [-0.01173472,  0.00531495, -0.01718942, ..., -0.02873163,
         0.00888342,  0.02952577],
       [ 0.0083612 ,  0.0075476 , -0.06756654, ..., -0.00699152,
         0.02319121,  0.00971042],
       [ 0.02860949, -0.03696719, -0.01945217, ...,  0.04612613,
        -0.06429625,  0.04685292]], dtype=float32)

In [9]:
preds

[' 방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요? 방청 페인트는 내구성이 상대적으로 떨어져서 실외 사용이나 강한 마모에 노출될 경우 추가적인 보호코팅이 필요할 수 있습니다.',
 ' 도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요? 도배지 안쪽의 길 형태의 타원으로 발생한 기포는 보수가가능합니다. 벽지면과 벽지면 사이가 벌어진 경우, 이것이 발생한 원인을 파악하기 위해 전문가의 조언을 받아야 합니다.',
 ' 큐블럭의 단점을 알려주세요. 또한, 압출법 단열판을 사용하는 것의 장점은 무엇인가요? 큐블럭은 시간이 지나면 단열 성능이 떨어지는 경시현상이 있을 수 있고, 비드법 단열재에 비해 가격이 상대적으로 높다는 것입니다. 이러한 특징으로 인해 건축 및 인테리어 분야에서 많은 사랑을 받고 있습니다.',
 ' 철골구조를 사용하는 고층 건물에서, 단열 효과를 높이기 위한 시공 방법은 무엇이 있을까요? 철골구조는 화재 발생 시 내구성이 약해지며, 이는 건물의 붕괴로 이어질 수 있습니다. 이러한 문제를 해결하기 위해 벽돌, 콘크리트 또는 석고보드와 같은 내화성 구조물을 둘러싸는 것이 좋습니다. 또한, 화재 감지 및 화재 진압 장치를 설치하여 초기에 화재를 진압하고 피난로를 확보하는 것이 중요합니다. 불꽃이 건물 구조물에 접근하지 못하게 방화벽 및 방화구조물을 설치하고, 발생 가능한 화재 경로에 화재 방지 장치를 설치하는 등의 화재 대응 시스템을 구축하는 것이 필요합니다.',
 ' 도배지의 완전한 건조를 위해 몇 주 동안 기다려야 하나요? 도배 작업을 위해서는 충분한 조명과 함께 적절한 작업을 위한 준비가 필요합니다.',
 ' 철근철골콘크리트 구조가 적용된 건물의 장단점은 무엇인가요? 철근철골콘크리트는 철골과 인장강도를 보유하여 강도와 내구성을 보유하고 있습니다. 또한 화재에 취약하고 녹이 발생하는 것을 방지하기 위해 내화재를 사용하고 콘크리트 표면을 벽과지 사이에 단열재로 도막이

In [19]:
submit = pd.read_csv('/content/drive/MyDrive/DBDBDeep/sample_submission.csv')
# 제출 양식 파일(sample_submission.csv)을 활용하여 Embedding Vector로 변환한 결과를 삽입
submit.iloc[:,1:] = pred_embeddings
submit.head()

,id,vec_0,vec_1,vec_2,vec_3,vec_4,vec_5,vec_6,vec_7,vec_8,...,vec_502,vec_503,vec_504,vec_505,vec_506,vec_507,vec_508,vec_509,vec_510,vec_511
0,TEST_000,0.020609,0.055621,0.016142,-0.062968,0.097745,0.026443,0.023927,0.010208,0.017878,...,-0.034149,-0.024314,0.020904,-0.015879,0.009484,0.042119,0.023135,-0.027046,-0.037981,0.068445
1,TEST_001,-0.013309,-0.009078,-0.003749,0.002257,0.070888,-0.047492,0.001522,0.001473,-0.036241,...,-0.024665,-0.033805,0.045180,-0.041855,-0.017163,0.055470,0.012837,-0.035813,0.008289,0.059890
2,TEST_002,0.032599,-0.048046,-0.074747,-0.023893,0.098163,-0.055020,-0.004950,-0.009477,0.046166,...,-0.045533,-0.000487,0.056645,-0.033378,0.022261,-0.010446,-0.025902,-0.011132,-0.038995,0.080175
3,TEST_003,0.030638,-0.015844,-0.019721,0.031987,0.058525,-0.056439,-0.055738,0.015962,0.024054,...,-0.012307,0.028784,0.060103,-0.058331,0.010810,-0.001387,-0.011752,-0.022528,-0.031257,0.082873
4,TEST_004,-0.006434,-0.006002,-0.003122,-0.013688,0.055164,-0.011476,0.011644,0.046652,0.011721,...,0.015999,-0.023894,0.083989,-0.018943,-0.028649,-0.020325,-0.020299,0.033385,-0.058084,-0.004951


In [16]:
pred_embeddings

array([[ 0.02060867,  0.05562091,  0.01614225, ..., -0.02704569,
        -0.03798091,  0.0684455 ],
       [-0.01330859, -0.0090784 , -0.00374858, ..., -0.03581328,
         0.00828904,  0.05989039],
       [ 0.03259891, -0.04804637, -0.0747473 , ..., -0.01113238,
        -0.03899511,  0.08017483],
       ...,
       [-0.01173472,  0.00531495, -0.01718942, ..., -0.02873163,
         0.00888342,  0.02952577],
       [ 0.0083612 ,  0.0075476 , -0.06756654, ..., -0.00699152,
         0.02319121,  0.00971042],
       [ 0.02860949, -0.03696719, -0.01945217, ...,  0.04612613,
        -0.06429625,  0.04685292]], dtype=float32)

In [23]:
submit.to_csv('/content/drive/MyDrive/DBDBDeep/hansol_skt.csv', index=False)

In [24]:
s = pd.read_csv('/content/drive/MyDrive/DBDBDeep/hansol_skt.csv')
s.head()

,id,vec_0,vec_1,vec_2,vec_3,vec_4,vec_5,vec_6,vec_7,vec_8,...,vec_502,vec_503,vec_504,vec_505,vec_506,vec_507,vec_508,vec_509,vec_510,vec_511
0,TEST_000,0.020609,0.055621,0.016142,-0.062968,0.097745,0.026443,0.023927,0.010208,0.017878,...,-0.034149,-0.024314,0.020904,-0.015879,0.009484,0.042119,0.023135,-0.027046,-0.037981,0.068445
1,TEST_001,-0.013309,-0.009078,-0.003749,0.002257,0.070888,-0.047492,0.001522,0.001473,-0.036241,...,-0.024665,-0.033805,0.045180,-0.041855,-0.017163,0.055470,0.012837,-0.035813,0.008289,0.059890
2,TEST_002,0.032599,-0.048046,-0.074747,-0.023893,0.098163,-0.055020,-0.004950,-0.009477,0.046166,...,-0.045533,-0.000487,0.056645,-0.033378,0.022261,-0.010446,-0.025902,-0.011132,-0.038995,0.080175
3,TEST_003,0.030638,-0.015844,-0.019721,0.031987,0.058525,-0.056439,-0.055738,0.015962,0.024054,...,-0.012307,0.028784,0.060103,-0.058331,0.010810,-0.001387,-0.011752,-0.022528,-0.031257,0.082873
4,TEST_004,-0.006434,-0.006002,-0.003122,-0.013688,0.055164,-0.011476,0.011644,0.046652,0.011721,...,0.015999,-0.023894,0.083989,-0.018943,-0.028649,-0.020325,-0.020299,0.033385,-0.058084,-0.004951
